In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge/jupyter notebooks
Current path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge


In [2]:
import torch
import torch.utils.data as data
from torch.utils.data import ConcatDataset, DataLoader, Dataset, ConcatDataset
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import glob
import PIL.Image as Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
from ipywidgets import interact, fixed
from torchvision import transforms
import torchvision.models as models
from Models.ResNet3d import ResNet3d
from lit_models.ResNet3d_lit import ResNet3d_lit
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Scrolls_Dataset_Module import Scrolls_Dataset

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.ToPILImage()

# Dataset Modules

### CONFIGS

In [3]:


# scroll_1 size = 8181, 6330
# scroll_2 size = 14830, 9506
# scroll_3 size = 7606, 5249

In [4]:
# Initiating Dataset with parameters

# buffer =   -- x,y patchsize for training
# z_start =  --  Offset of slices in the z direction
# z_dim =    -- Number of slices in the z direction. Max value is (64 - z_start)
# validation_rect =  -- rectangle removed for validation set
# shared_height = -- Height to resize all scrolls
# scroll_fragments = -- scrolls to be used 

dataset = Scrolls_Dataset(
                buffer = 31,
                 z_start = 20,
                 z_dim = 24,
                 validation_rect = (1100, 3500, 700, 950),
                shared_height = 2000,
                 downsampling =None,
                 scroll_fragments = [1,2,3],
                 stage = 'train',
                 shuffle=True,
                 batch_size=8,
                 num_workers =4 ,
                 on_gpu= False,
                          
                         )



In [5]:
# prepeare data, by processng images and loading dataloader

#dataset.prepare_data()

In [6]:
model = ResNet3d(
             num_classes=2,
            embedding_dim=128,
            z_dim=24,
            buffer=31,

            )

/Users/gregory/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gregory/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
lit_model = ResNet3d_lit(model=model)

In [12]:
trainer = Trainer(accelerator='cpu', devices=1,  max_epochs=1,  accumulate_grad_batches=8) #accumulate_grad_batches=2, overfit_batches=1, default_root_dir="lightning_logs/ResNet/checkpoints"  
trainer.fit(model=lit_model, datamodule=dataset, )


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/gregory/opt/anaconda3/lib/python3.9/site-packages/PIL/Image.py:3035: DecompressionBombWarning: Image size (140973980 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(

0it [00:00, ?it/s]
6it [00:00, 52.48it/s]
13it [00:00, 57.91it/s]
24it [00:00, 57.04it/s]

0it [00:00, ?it/s]
7it [00:00, 68.87it/s]
14it [00:00, 64.26it/s]
24it [00:00, 64.96it/s]

0it [00:00, ?it/s]
7it [00:00, 69.25it/s]
15it [00:00, 73.20it/s]
24it [00:00, 74.17it/s]

  | Name    | Type             | Params
---------------------------------------------
0 | model   | ResNet3d         | 33.2 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
33.2 M    Trainable params
0         Non-trainable params
33.2 M    Total params
132.977   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

RuntimeError: Conv3D is not supported on MPS

In [ ]:
torch.randn((8,2)).view(-1).shape